# Set up

In [10]:
import duckdb
import pandas as pd
from IPython.display import display
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from eda_support_functions import *

# Suppress all warnings
import warnings

warnings.filterwarnings('ignore')

In [11]:
full__regression__none = pd.read_csv('../data/ml_ready_data/full__regression__with_outliers__none.csv')

In [12]:
full__regression__none

,movie_id,original_language,runtime,ageCert,quarter,month,year,is_released_US,is_released_CN,is_released_FR,is_released_GB,is_released_JP,budget_usd_adj,revenue_usd_adj,is_outlier,production_size
0,340666,en,117,R,4,11,2016,True,False,True,True,False,2.856498e+07,3.848256e+07,False,medium_productions
1,242911,te,168,U,4,12,2006,False,False,False,False,False,5.481741e+06,9.593046e+06,True,small_productions
2,179826,en,100,PG13,1,1,2013,True,False,False,False,True,3.531533e+07,1.503213e+06,True,large_productions
3,340676,fr,106,R,4,12,2016,True,False,True,True,False,7.617328e+06,3.606053e+06,False,small_productions
4,505058,en,93,R,3,7,2018,True,False,True,True,False,1.213435e+06,1.941496e+07,False,small_productions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12122,276478,en,95,PG13,2,4,1988,True,False,False,False,True,8.567825e+06,2.622214e+05,False,small_productions
12123,14342,en,90,PG13,2,5,1998,True,False,False,False,False,5.608012e+07,1.147185e+07,False,large_productions
12124,441894,hi,136,U,3,9,2017,False,False,False,False,False,1.528980e+07,3.356297e+07,True,medium_productions
12125,54948,pt,102,U,2,5,2001,False,False,False,False,False,1.032305e+06,6.225913e+04,True,small_productions


In [13]:
full__regression__none.shape

(12127, 16)

In [14]:
(full__regression__none
 .groupby(['is_outlier', 'production_size'])
 .size()
 .unstack(fill_value=0)
 .rename(index={False: 'Not Outlier', True: 'Outlier'})
 .assign(Total=lambda x: x.sum(axis=1))
 .pipe(lambda df: pd.concat([df, pd.DataFrame([df.sum()], index=['Total'])])
       .fillna(0)
       .astype(int))
 .style
#  .background_gradient(cmap='YlOrRd')
 .format("{:,d}")
)

production_size,large_productions,medium_productions,small_productions,Total
Not Outlier,"2,839","2,059","2,781","7,679"
Outlier,337,647,"3,464","4,448"
Total,"3,176","2,706","6,245","12,127"


In [15]:
full__regression__none['ratio'] = full__regression__none['budget_usd_adj'] / full__regression__none['revenue_usd_adj']

In [16]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=False)[['movie_id','original_language','year','ratio', 'budget_usd_adj','revenue_usd_adj']].head(450).to_clipboard(index=False))

None

In [17]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=False)[['movie_id','original_language','year','ratio', 'budget_usd_adj','revenue_usd_adj']].head(450).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,original_language,year,ratio,budget_usd_adj,revenue_usd_adj
11720,13842,en,2006,43.689802,"$16,625,605","$380,537"
152,33932,en,2003,43.677913,"$49,679,674","$1,137,410"
4443,20542,en,2008,43.675751,"$56,608,965","$1,296,119"
638,35161,en,1997,43.583410,"$2,060,999","$47,289"
956,376873,hi,2016,43.569223,"$3,832,618","$87,966"
5131,42230,zh,1977,43.373087,"$3,787,676","$87,328"
10216,51333,en,2000,43.333333,"$23,003,055","$530,840"
6522,45225,en,1989,42.566595,"$12,286,371","$288,639"
3971,40374,en,1973,42.246381,"$5,146,993","$121,833"
7973,47439,fr,1999,41.765223,"$19,021,013","$455,427"


In [18]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=True)[['movie_id', 'ratio', 'budget_usd_adj','revenue_usd_adj']].head(10).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,ratio,budget_usd_adj,revenue_usd_adj
11087,13120,0.019317,"$36,739","$1,901,872"
3302,2108,0.019408,"$2,831,803","$145,909,132"
8473,27420,0.019960,"$838,014","$41,984,517"
3269,39154,0.020000,"$1,699,082","$84,954,089"
2847,1781,0.020088,"$1,511,419","$75,241,461"
3624,19053,0.020180,"$1,070,740","$53,058,518"
776,80220,0.020535,"$2,333,519","$113,634,739"
2848,1779,0.020762,"$393,164","$18,936,659"
3424,2255,0.020796,"$474,614","$22,821,842"
3673,19108,0.020833,"$1,257,021","$60,337,030"
